In [57]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import sentencepiece
# !pip install transformers
# !pip install sentencepiece
# !pip install torchvision==0.17.0
# !pip install --upgrade torch

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

In [58]:
# Pharmacy database, ground truth
def get_pharmacy_info():
    return {
        "medications": ["Aspirin", "Ibuprofen", "Paracetamol"],
        "prices": {"Aspirin": 5.0, "Ibuprofen": 7.5, "Paracetamol": 3.0},
        "stock": {"Aspirin": 50, "Ibuprofen": 30, "Paracetamol": 100},
        "indications": {"Aspirin": "headache", "Ibuprofen": "pain relief", "Paracetamol": "fever"},
        "dosages": {"Aspirin": "500mg twice daily", "Ibuprofen": "400mg every 6 hours", "Paracetamol": "500mg every 4-6 hours"}
    }

# Build context
def generate_context(info):
    lines = []
    for m in info["medications"]:
        lines.append(
            f"{m}: cost={info['prices'][m]} euros, stock={info['stock'][m]}, "
            f"indication={info['indications'][m]}, dosage={info['dosages'][m]}"
        )
    return "Inventory:\n" + "\n".join(lines)

def ask_question(question, context):
    instruction = (
        "You are a pharmacist assistant. Use only the information in the context. "
        "Answer concisely with only the requested data."
    )
    prompt = f"{instruction}\nContext: {context}\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").input_ids
    output = model.generate(inputs, max_length=16)
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()


def generate_followup_prompt(question, info):
    q = question.lower()
    matched_meds = [m for m in info['medications'] if m.lower() in q]
    if len(matched_meds) == 1 and matched_meds[0] in info['dosages']:
        return f"What is the recommended dosage for {matched_meds[0]}?"
    return None

In [61]:
pharmacy_info = get_pharmacy_info()
context = generate_context(pharmacy_info)

print("=== Pharmacy Chatbot ===")
print("Ask about cost, stock, indications, or dosage.")
print()

questions = [
    "What does Aspirin cost?",
    "How many Ibuprofen are in stock?",
    "Which medications cost more than 4 euros?"
]

for q in questions:
    print(f"## Question: {q}\n")
    ans = ask_question(q, context)
    print(f"Answer: {ans}")
    followup = generate_followup_prompt(q, pharmacy_info)
    if followup:
        print(f"\nFollow-up: {followup}")
        follow_ans = ask_question(followup, context)
        print(f"Follow-up answer: {follow_ans}")
    print()


=== Pharmacy Chatbot ===
Ask about cost, stock, indications, or dosage.

## Question: What does Aspirin cost?

Answer: 5.0 euros

Follow-up: What is the recommended dosage for Aspirin?
Follow-up answer: 500mg

## Question: How many Ibuprofen are in stock?

Answer: 50

Follow-up: What is the recommended dosage for Ibuprofen?
Follow-up answer: 400mg

## Question: Which medications cost more than 4 euros?

Answer: Paracetamol



In [51]:
# Check for incorrect indications
def safety_check(question, info):
    q = question.lower()
    for m in info['medications']:
        if m.lower() in q and 'against' in q:
            asked = q.split('against')[1].strip().rstrip('?')
            actual = info['indications'][m].lower()
            if asked != actual:
                return f"!!! Security check: {m} is indicated for {info['indications'][m]}, not for {asked}."
    return None

questions = [
    "What are the costs for Aspirin against diarrhea?",
    "What are the costs for Paracetamol against fever?",
    "What are the costs for Paracetamol against a cold?"
]

for q in questions:
    print(f"## Question: {q}\n")
    err = safety_check(q, pharmacy_info)
    if err:
        print(err)
    else:
        ans = ask_question(q, context)
        print(f"Answer: {ans}")
        followup = generate_followup_prompt(q, pharmacy_info)
        if followup:
            print(f"\nFollow-up: {followup}")
            follow_ans = ask_question(followup, context)
            print(f"Follow-up answer: {follow_ans}")
    print()

## Question: What are the costs for Aspirin against diarrhea?

!!! Security check: Aspirin is indicated for headache, not for diarrhea.

## Question: What are the costs for Paracetamol against fever?

Answer: 7.5 euros

Follow-up: What is the recommended dosage for Paracetamol?
Follow-up answer: 500mg

## Question: What are the costs for Paracetamol against a cold?

!!! Security check: Paracetamol is indicated for fever, not for a cold.

